<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;"> AutoEDA + Feature Engineering </h1><a id=0></a>


![Pak Wheels](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.pakwheels.com%2Fblog%2Ftoyota-revises-vehicle-prices%2F&psig=AOvVaw1MlSB61yiCQC8x7-zilzkd&ust=1638113079737000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCOjUtPTsuPQCFQAAAAAdAAAAABAD)

# **Context**

PakWheels is the largest online marketplace for car shoppers and sellers in Pakistan. It aggregates thousands of new, used, and certified second-hand cars from thousands of dealers and private sellers. The site also provides users with automotive reviews, shopping advice, and comparison tools for car financing and insurance information. It serves as an online community of automobile enthusiasts, with over 250,000 registered members on its online forum.

# **Content**
What's inside is more than just rows and columns.

**@type:** Vehicle type

**model:** Vehicle model

**description:** Ad/vehicle description

**itemCondition:** vehicle condition

**modelDate:** vehicle model release date

**manufacturer:** vehicle manufacturer

fuelType vehicle fuel type

**name:** Ad title

**image:** ad/vehicle cover image

**vehicleTransmission:** vehicle transmission type(manual, automatic)

**color:** vehicle color

**bodyType:** vehicle body type

**mileageFromOdometer:** km's the vehicle has driven

**sellerLocation:** seller location

**postedFrom:** platform ad posted from

**keywords:** ad search keywords

**features:** list of features available in the car. e.g: AC, Power windows, etc.

**adLastUpdated:** Ad last updated date

**price:** Asking price

**priceCurrency:** Currency. e.g: PKR, USD.

**brand:{**

**@type:**

**name:** vehicle brand name
}

**vehicleEngine:**{

**@type:**

**engineDisplacement:** km's the vehicle has driven

}

**extraFeatures:** {

**RegisteredIn:** city the vehicle is registered in.

**Color:** vehicle color

**Assembly:** vehicle assembly location(local, imported

**EngineCapacity:** engine capacity in cubic centimeters(cc)

**BodyType:** vehicle body type

**LastUpdated:** Ad last updated

**AdRef#:** Ad reference number

**AuctionGrade:** a detailed explanation on auction grade

**ChassisNo.:**

**ImportDate:** vehicle import date

**BatteryCapacity:** vehicle battery capacity

**Warranty:** warranty in combination of time duration and distance traveled

}


<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;"> Importing Libraries </h1><a id=0></a>


In [1]:
!pip install dataprep
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from dataprep.eda import plot,plot_correlation,plot_missing,plot_diff,create_report
import os
import json

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  First Look at Data </h1><a id=0></a>


In [2]:
pd.set_option('display.max_columns',500)
with open('../input/pakistans-largest-pakwheels-automobiles-listings/usedCars.json') as f:
    DICT = json.load(f)



data = pd.json_normalize(DICT['usedCars'])
data.head()

In [3]:
data.shape

**The data has 55675 rows and 37 columns**

In [4]:
data.info()

Only three features are having integer datatype.

Later in this notebook we are going to convert mileageFromodometer and Engine Capacity to integer datatype.

This will help us understand the relationship between price and these features.

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  First Visualization Using AutoEDA </h1><a id=0></a>


In [5]:
plot(data)

Through DataPrep library, it took us few seconds to visualize all features of the data.
The plot() function explores the distributions and statistics of the dataset. It generates a variety of visualizations and statistics which enables us to achieve a comprehensive understanding of the column distributions and their relationships.

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  Bit of Feature Engineering </h1><a id=0></a>


As we saw from the plot() function that are many columns which are not going to help us visualize and extract insights from data very well. So we are going to drop those columns.

**Reason for dropping these columns:**

1. Some of them have constant values such as @type feature. We should remove all these kind of features to reduce memory as they are not going to help us extract usefull insights from data.

2. Many of the columns contain same data,specially those columns which start with 'extraFeatures'. For example extraFeatures.color. We already have a column name as Color. So we dont need it.

3. Many of the columns starting with 'extraFeatures' contain 99% missing values. So its better to remove them and make our data tidy.


In [6]:
remove1=['@type','description','itemCondition','name','image','brand.@type','brand.name','keywords','adLastUpdated',
                'priceCurrency','vehicleEngine.@type','vehicleEngine.engineDisplacement']
not_remove=['extraFeatures.Assembly','extraFeatures.RegisteredIn','extraFeatures.EngineCapacity']

remove2=[col for col in data.columns if col not in not_remove and 'extraFeatures' in col]
drop_columns=remove1+remove2
data.drop(drop_columns,axis=1,inplace=True)
                

Lets have a look at features now. 


In [7]:
data.columns

Now it looks more better and cleaner. We have removed redundant features from our dataset.

Next we will convert EngineCapacity and MileagefromOdometer to integer datatypes. They are going to help us relate with price column

In [8]:
data['extraFeatures.EngineCapacity']

We will remove 'cc' from every sample and then convert them into integer datatype.

In [9]:
data['extraFeatures.EngineCapacity']=data['extraFeatures.EngineCapacity'].str.replace('cc','').str.strip()
data.dropna(subset=['extraFeatures.EngineCapacity'],inplace=True)
data['extraFeatures.EngineCapacity']=data['extraFeatures.EngineCapacity'].astype('int64')

In [10]:
data['extraFeatures.EngineCapacity']

Now its clean. 

Next turn is of mileageFrom Odometer. We will do same tasks. But this time we will remove 'km' from every sample and then convert them to integer datatype.

In [11]:
data.mileageFromOdometer

In [12]:
data['mileageFromOdometer']=data['mileageFromOdometer'].str.replace(',','')
data['mileageFromOdometer']=data['mileageFromOdometer'].str[:-2].str.strip()
data['mileageFromOdometer']=data['mileageFromOdometer'].astype('int64')

In [13]:
data['mileageFromOdometer']

It looks good.

Now its time for sellerLocation feature. Lets first have a look at some of its values.

In [18]:
data.sellerLocation

This feature is very important as it tells us the location of the seller/where the car is.

As you can see that it contains exact address in city. This way there would be far too many values to look at. So we will just extract the city name and the province from the location. 

In [19]:
data['seller_city']=(data['sellerLocation'].str.split().str[-2:]).str.join(' ')
data['seller_city']

In [21]:
data.drop('sellerLocation',axis=1,inplace=True)

We have successfully cleaned it and stored them in a new feature named as 'seller_city'.


<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  Univariate Analysis </h1><a id=0></a>


We will have a look at each feature of data to have an idea about it. We are going to use autoEDA framework DataPrep for it.

In [22]:
columns=[col for col in data.columns ]
for column in columns:
    print(f'\t\t\t\t {column}')
    plot(data,column).show()
    print('------------------------------------------------------------------------------------------------------------------')
   

Through these visualizations we know the following characterstics of each feature.

**model:** Corolla is the most sold car in this data. The largest value (Corolla) is over 1.9 times larger than the second largest value (Civic). It takes 18.3% of car models.

**modelDate:** It is the model year of car. It has 1888 outliers and is left skewed. 12.49% of the car model years range from (2014-2016).

**manufacturer:** Toyota,Suzuki and Honda are leading manufacturers respectively. They represent 84% of the total manufacturers.

**fuelType:** Petrol is the leading fuel type in all cars. The largest value (Petrol) is over 21.78 times larger than the second largest value (Diesel).

**vehicleTransmission:** Approximately 53% of cars are automatic and 57% of cars are manual.

**colors:** White, silver, black and grey are leading colors. They together correspond to 80.7% of colors. The largest value (White) is over 2.58 times larger than the second largest value (Silver).

**bodyType:** Sedan and batchback are leading body types. They represent 71% of total body types. It also contains 12.5% of missing data.

**mileageFromOdometer:** It is right skewed with 17.36% of it ranges from 80000km to 100000km.

**price:** It is right skewed with approx 56% ranges from PKR 98000 to PKR 1900000.

**extraFeatures.RegisteredIn:** Many cars have registeration from Lahore,Islamabad and Karachi representing 71% of the registeration cities.

**extraFeatures.Assembly:** 2/3 rd of cars are locally manufactured.

**extraFeatures.EngineCapacity:** It is also right skewed with approx 21% ranges from 1280cc to 1400cc.

**seller_city:** Majority of cars are location in Lahore, Karachi, Islamabad and Rawalpindi.

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  Missing Data </h1><a id=0></a>


Lets use DataPrep to have visualizations for missing data.

In [29]:
plot_missing(data)

Only bodytype has missing data totalling to 12.48%

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  Correlations with Price </h1><a id=0></a>


Nows its time to check the correlations between price and other features column.

In [30]:
plot_correlation(data)

extraFeatures.Capacity is most positvely correlated with price according to pearson correlation of 0.63.

modelDate also has positive pearson correlation of 0.28 with price.

mileage featurs has negative perason correlation of -0.17 with price.

The following plots elaborate this.

In [31]:
plot_correlation(data,'modelDate','price')

In [32]:
plot_correlation(data,'mileageFromOdometer','price')

In [33]:
plot_correlation(data,'extraFeatures.EngineCapacity','price')

In [34]:
plot_correlation(data,'modelDate','price')

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  Bivariate Analysis </h1><a id=0></a>


In this section we will compare some features with price feature to have a better understanding of them.


In [35]:
plot(data,'model','price')

Here prado seems to have the highest median price.
Lets plot a bar graph to confirm this.


In [36]:
data.groupby('model')['price'].median().nlargest(10).plot.bar()


Here we tried to have an idea that which car model is most expensive and contrary to my expectations g class is ranked on top. that is because it only has 2 rows and so its median ranks highest.

In [37]:
data[data['model']=='G Class']

To have a better understanding of data, in next cell i will create a new dataframe where i will filter rows on the basis of each models value count. if a model's value count is greater than 100, then we will consider it, else it will be filtered out. this way we will get better plot and analyze it well.

In [38]:
df1=data[data['model'].map(data['model'].value_counts())>100]

In [39]:
df1.groupby('model')['price'].median().nlargest(10).plot.bar()

Finally, our plot ssuggests that land cruisers and prado have highest median price.


Now lets compare manufacturer and price features.

In [40]:
plot(data,'manufacturer','price')

There seem to have many outliers in some manufacturers.

Lets get an idea of the most expensive manufacturer.

In [41]:
data.groupby('manufacturer')['price'].median().nlargest(10).plot.bar()


The plot shows that manufacturer called Bentley is most expensive. But I dont think so. 

Lets confirm this by looking at its sample.

In [42]:
data[data['manufacturer']=='Bentley']

So it contains only one sample thats why the plot showed that it was most expensive manufacturer. 

Lets make it clear by what we did with models.

In [43]:
df1=data[data['manufacturer'].map(data['manufacturer'].value_counts())>100]

In [44]:
df1.groupby('manufacturer')['price'].median().nlargest(10).plot.bar()


Finally, audi seems to be most expensive manufacturer.

Lets find relation between bodytype and price features.

In [45]:
plot(data,'bodyType','price')

COMPACT SUV seems to have highest median price. lets confirm this.

In [46]:
data.groupby('bodyType')['price'].median().nlargest(10).plot.bar()


In [47]:
df1=data[data['bodyType'].map(data['bodyType'].value_counts())>50] 

In [48]:
df1.groupby('bodyType')['price'].median().nlargest(10).plot.bar()


Compact SUV has highest median price.

In [49]:
plot(data,'fuelType','price')

In [50]:
data.groupby('fuelType')['price'].median().nlargest(10).plot.bar()


Hybrid cars have highest median price.

In [51]:
plot(data,'seller_city','price')

In [52]:
data.groupby('seller_city')['price'].mean().nlargest(10) .plot.bar()

In [53]:
data['seller_city'].value_counts(ascending=True).nsmallest(10)

In [54]:
df1=data[data['seller_city'].map(data['seller_city'].value_counts())>100]

In [55]:
df1.groupby('seller_city')['price'].mean().nlargest(10).plot.bar()

In [56]:
plot(data,'extraFeatures.RegisteredIn','price')

In [57]:
data.groupby('extraFeatures.RegisteredIn')['price'].mean().nlargest(10).plot.bar()

In [58]:
data[data['extraFeatures.RegisteredIn']=='Carbon']

In [59]:
df2=data[data['extraFeatures.RegisteredIn'].map(data['extraFeatures.RegisteredIn'].value_counts())>100]
df2.groupby('extraFeatures.RegisteredIn')['price'].median().nlargest(10).plot.bar()

In [60]:
plot(data,'extraFeatures.Assembly','price')

Imported cars have lots of outliers.

In [61]:
data.groupby('extraFeatures.Assembly')['price'].median().nlargest(10).plot.bar()

Imported cars are more expensive than local cars.

<h1 style="background-color:#FFA500
;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">  Conclusion </h1><a id=0></a>


1- Majority of the cars are from big cities. Imported Cars are most expensive.

2- The more latest model of the car is, more expensive it is.

3- The more engine capacity the car has, more expensive it is.

4- The more mileage the car has, cheaper the car gets.

5- Audi manufactures the most expensive car.

6- Compact SUV body type cars are more expensive.

7- Diesel cars are more expensive than others.
